In [1]:
import segmentation_models as sm
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from fetch_data import fetch
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

2023-02-15 00:07:37.891015: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-15 00:07:37.983879: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-15 00:07:37.986421: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-15 00:07:37.986435: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

Segmentation Models: using `tf.keras` framework.


In [2]:
base_dir = '/media/dev/WinD/Curious Dev B/PROJECT - 1/COVID-19 for segmentation/'
batch_size = 16
input_size = (224, 224)
num_epochs = 5
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

callbacks = [ModelCheckpoint('./lung_best_model.h5', 
                             save_weights_only=True, 
                             save_best_only=True, 
                             mode='min'),
            ReduceLROnPlateau(monitor = "val_iou_score",
                             factor = 0.2,
                             patience = 5)]

In [3]:
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()
    
def denormalize(x):
    x_max = np.percentile(x, 98)
    x_min = np.percentile(x, 2)    
    x = (x - x_min) / (x_max - x_min)
    x = x.clip(0, 1)
    return x

In [4]:
train_dataset, validation_dataset = fetch(base_dir, input_size)

train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=1024)

validation_dataset = validation_dataset.batch(1)

2023-02-15 00:07:39.581944: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-15 00:07:39.581974: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-15 00:07:39.581988: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Turbo): /proc/driver/nvidia/version does not exist
2023-02-15 00:07:39.582237: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/dev/anaconda3/lib/python3.9/site-packages/tensorflow/python/dat

In [5]:
model = sm.Unet(backbone_name = "efficientnetb4",
                   input_shape=(224, 224, 3))

model.compile(optimizer=Adam(learning_rate = 0.001), 
              loss='binary_crossentropy', 
              metrics=metrics)

71892840/71892840 [==============================] - 32s 0us/step


In [ ]:
history = model.fit(train_dataset, epochs=num_epochs,
                    validation_data=validation_dataset,
                    callbacks=callbacks)

Epoch 1/5
75/75 [==============================] - 672s 9s/step - loss: 0.1819 - iou_score: 0.8519 - f1-score: 0.9110 - val_loss: 0.9116 - val_iou_score: 0.5762 - val_f1-score: 0.7249 - lr: 0.0010
Epoch 2/5
75/75 [==============================] - 674s 9s/step - loss: 0.0573 - iou_score: 0.9298 - f1-score: 0.9635 - val_loss: 0.0792 - val_iou_score: 0.9043 - val_f1-score: 0.9495 - lr: 0.0010
Epoch 3/5
75/75 [==============================] - 674s 9s/step - loss: 0.0431 - iou_score: 0.9429 - f1-score: 0.9705 - val_loss: 0.0431 - val_iou_score: 0.9355 - val_f1-score: 0.9665 - lr: 0.0010
Epoch 4/5
75/75 [==============================] - 675s 9s/step - loss: 0.0376 - iou_score: 0.9478 - f1-score: 0.9731 - val_loss: 0.0368 - val_iou_score: 0.9420 - val_f1-score: 0.9700 - lr: 0.0010
Epoch 5/5
22/75 [=======>......................] - ETA: 6:55 - loss: 0.0338 - iou_score: 0.9532 - f1-score: 0.9760

In [ ]:
model.load_weights('lung_best_model.h5') 

In [ ]:
scores = model.evaluate(validation_dataset)

print("Loss: {:.5}".format(scores[0]))
for metric, value in zip(metrics, scores[1:]):
    print("mean {}: {:.5}".format(metric.__name__, value))

In [ ]:
for batch in validation_dataset:
    image, gt_mask = batch
    image = image.numpy()
    gt_mask = gt_mask.numpy()
    pr_mask = model.predict(image).round()
    
    visualize(
        image=denormalize(image.squeeze()),
        gt_mask=gt_mask[..., 0].squeeze(),
        pr_mask=pr_mask[..., 0].squeeze(),
    )

In [ ]:
# Resize all images to 224, 224. 
# Convert all images to grayscale